In [79]:
import glob
import re
import pandas as pd
benchmarks = glob.glob("../benchmarks-new/*.fz")

In [80]:
names = [x.split(".")[2].split("/")[-1] for x in benchmarks if "factor" not in x]
names

['kepler0',
 'sum',
 'test01_sum3',
 'shoelace-formula',
 'kepler1',
 'rigidBody2',
 'matrixDeterminant2',
 'sineOrder3',
 'delta4',
 'matrixDeterminant',
 'add-assoc',
 'rigidBody1',
 'himmilbeau',
 'sqroot']

In [81]:
def parse_bound(bnd: str):
    bnd = bnd.strip("[]")
    # print("bnd", bnd)
    bnd = bnd.strip()

    if "b" in bnd:
        mantissa = int(bnd.split("b")[0])
        exp = int(bnd.split("b")[1])
        # print("mantissa, exp)
        return mantissa * (2 ** exp)
    else:
        return float(bnd)
    # print(left, right)

In [82]:
def parse_gappa_out(filename):
    try:
        with open(filename, "r") as f:
            file = f.read()
        if "Error" in file:
            return None
            
        file = re.sub("\{.*?\}", "", file).strip()
        # try:
        bounds = [parse_bound(x.strip()) for x in file.split("in")[1].strip().split(",")]
        # except as e:
        #     print(file, e)
            # try:
            # except:
            #     print(filename, ":", file)
            
        lb = float(bounds[0])
        ub = float(bounds[1])
        return (lb, ub)
        
    except:
        (0,0)

<>:8: SyntaxWarning: invalid escape sequence '\{'
<>:8: SyntaxWarning: invalid escape sequence '\{'
/var/folders/_8/qzz3btp16yl2y5nkqx7g0v0c0000gp/T/ipykernel_62139/3921668004.py:8: SyntaxWarning: invalid escape sequence '\{'
  file = re.sub("\{.*?\}", "", file).strip()


In [83]:
results = []
for fname in names:
    with open(f"../benchmarks-new/{fname}.fz.out") as f:
        lines = str(f.read()).split("\n")
        # print(lines)
        pre_rel = lines[0].split(":")[1].strip().replace("Some", "").replace("(", "").replace(")", "")
        pre_abs = lines[1].split(":")[1].strip()

    with open(f"../benchmarks-new/{fname}-factor.fz.out") as f:
        lines = str(f.read()).split("\n")
        # print(lines)
        pre_rel_factor = lines[0].split(":")[1].strip().replace("Some", "").replace("(", "").replace(")", "")
        pre_abs_factor = lines[1].split(":")[1].strip()
        
    g_abs = parse_gappa_out(f"../benchmarks-new/{fname}-abs.g.out")
    if g_abs:
        g_abs = g_abs[1]
    else:
        g_abs = None
    g_rel = parse_gappa_out(f"../benchmarks-new/{fname}-rel.g.out")
    if g_rel:
        g_rel = g_rel[1]
    else:
        g_rel = None
    # print("PairFuzz", fname, pre_rel, pre_abs)
    # print("PairFuzz (factor)", fname + "-factor", pre_rel_factor, pre_abs_factor)
    # print("Gappa", g_abs, g_rel)
    result = {
        "benchmark": fname,
        "pre-rel": pre_rel,
        "pre-abs": pre_abs,
        "pre-rel-factor": pre_rel_factor,
        "pre-abs-factor": pre_abs_factor,
        "gappa-abs": g_abs,
        "gappa-rel": g_rel,
    }
    results.append(result)
    print(result)

{'benchmark': 'kepler0', 'pre-rel': 'None', 'pre-abs': '9.53407663928374e-13', 'pre-rel-factor': 'None', 'pre-abs-factor': '6.810054742345528e-13', 'gappa-abs': 1.2317258324401337e-13, 'gappa-rel': None}
{'benchmark': 'sum', 'pre-rel': 'None', 'pre-abs': '2.6645352591003757e-14', 'pre-rel-factor': 'None', 'pre-abs-factor': '1.3322676295501878e-14', 'gappa-abs': 3.6637359812630166e-15, 'gappa-rel': None}
{'benchmark': 'test01_sum3', 'pre-rel': 'None', 'pre-abs': '1.4305118838819908e-5', 'pre-rel-factor': 'None', 'pre-abs-factor': '7.1525583962284145e-6', 'gappa-abs': 1.9669532775878906e-06, 'gappa-rel': None}
{'benchmark': 'shoelace-formula', 'pre-rel': 'None', 'pre-abs': '3.996806885453452e-9', 'pre-rel-factor': 'None', 'pre-abs-factor': '1.998403442726726e-9', 'gappa-abs': None, 'gappa-rel': None}
{'benchmark': 'kepler1', 'pre-rel': 'None', 'pre-abs': '5.944255235590392e-12', 'pre-rel-factor': 'None', 'pre-abs-factor': '2.842904677891057e-12', 'gappa-abs': 5.381494361245132e-13, 'gapp

In [84]:
pd.DataFrame(results)

,benchmark,pre-rel,pre-abs,pre-rel-factor,pre-abs-factor,gappa-abs,gappa-rel
0,kepler0,None,9.53407663928374e-13,None,6.810054742345528e-13,1.231726e-13,NaN
1,sum,None,2.6645352591003757e-14,None,1.3322676295501878e-14,3.663736e-15,NaN
2,test01_sum3,None,1.4305118838819908e-5,None,7.1525583962284145e-6,1.966953e-06,NaN
3,shoelace-formula,None,3.996806885453452e-9,None,1.998403442726726e-9,NaN,NaN
4,kepler1,None,5.944255235590392e-12,None,2.842904677891057e-12,5.381494e-13,NaN
5,rigidBody2,None,2.0353718710452995e-10,None,1.0176859355226497e-10,3.606626e-11,NaN
6,matrixDeterminant2,None,2.2648549702353193e-11,None,6.661338147750939e-12,3.502976e-12,NaN
7,sineOrder3,None,5.937035405398565e-15,None,4.2407393152643635e-15,8.886006e-16,NaN
8,delta4,None,1.1438511683081742e-12,None,5.004348861348262e-13,1.394329e-13,NaN
9,matrixDeterminant,None,2.2648549702353193e-11,None,6.661338147750939e-12,3.502976e-12,NaN
